In [2]:
import torch
import numpy as np
import torch.nn as nn
from torch.autograd import Variable
import torchvision.models as models

In [3]:
def summarize_model(model):
    layers = [(name if len(name) > 0 else 'TOTAL', str(module.__class__.__name__), sum(np.prod(p.shape) for p in module.parameters())) for name, module in model.named_modules()]
    layers.append(layers[0])
    del layers[0]

    columns = [
        [" ", list(map(str, range(len(layers))))],
        ["Name", [layer[0] for layer in layers]],
        ["Type", [layer[1] for layer in layers]],
        ["Params", [layer[2] for layer in layers]],
    ]

    n_rows = len(columns[0][1])
    n_cols = 1 + len(columns)

    # Get formatting width of each column
    col_widths = []
    for c in columns:
        col_width = max(len(str(a)) for a in c[1]) if n_rows else 0
        col_width = max(col_width, len(c[0]))  # minimum length is header length
        col_widths.append(col_width)

    # Formatting
    s = "{:<{}}"
    total_width = sum(col_widths) + 3 * n_cols
    header = [s.format(c[0], l) for c, l in zip(columns, col_widths)]

    summary = " | ".join(header) + "\n" + "-" * total_width
    for i in range(n_rows):
        line = []
        for c, l in zip(columns, col_widths):
            line.append(s.format(str(c[1][i]), l))
        summary += "\n" + " | ".join(line)

    return summary

In [4]:
net = models.convnext_tiny(pretrained=True)
last_layer = net.classifier._modules["2"]
in_features = net.classifier._modules["2"].in_features
net.classifier._modules["2"] = nn.Sequential(
    nn.Linear(in_features, 4096),
    nn.ReLU(),
    nn.Linear(4096, 4096),
    nn.ReLU(),
    nn.Linear(4096, in_features),
    nn.ReLU(),
    last_layer,
)
num_trainable_params = sum(p.numel() for p in net.parameters() if p.requires_grad) / 1e6
print(f'Number of traininable params: {num_trainable_params:.2f}M')  # expected output: ~16M

Number of traininable params: 51.67M


In [7]:
net_1 = models.alexnet(pretrained=True).features
net_2 = models.alexnet(pretrained=True).classifier
net_2._modules["6"] = nn.Linear(4096, 40)
num_trainable_params = sum(p.numel() for p in net_1.parameters() if p.requires_grad) / 1e6
num_trainable_params2 = sum(p.numel() for p in net_2.parameters() if p.requires_grad) / 1e6
print(f'Number of traininable params: {num_trainable_params+num_trainable_params2:.2f}M')  # expected output: ~16M

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /home/ingo/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100.0%


Number of traininable params: 57.17M
